In [7]:
from torchvision.datasets import MovingMNIST, Kinetics
import os

In [1]:
from vitssm.data import build_dataset
from addict import Dict
import kappaconfig as kc
import os
from pathlib import Path

config = kc.from_file_uri(Path("configs") / "base_config.yml")
print(config.keys())
res = kc.DefaultResolver()
config = Dict(res.resolve(config))

dataset = build_dataset(config)

dict_keys(['root_dir', 'experiment', 'dataset', 'optimization', 'model'])
/mnt/c/Users/david/Desktop/ViTSSM/data


  5%|▌         | 45/833 [00:30<08:47,  1.49it/s]


KeyboardInterrupt: 

In [2]:
Path('/mnt/c/Users/david/Desktop/ViTSSM/data/UCF-101')

PosixPath('/mnt/c/Users/david/Desktop/ViTSSM/data/UCF-101')

In [20]:
print(os.getcwd())
os.chdir()

/mnt/c/Users/david/Desktop/ViTSSM


In [5]:
dataset

NameError: name 'dataset' is not defined

In [8]:
train_dataset[0].permute(0, 2, 3, 1).repeat(1, 1, 1, 3).shape

torch.Size([10, 64, 64, 3])

In [9]:
from torchvision.io import write_video

write_video("test.mp4", train_dataset[0].permute(0, 2, 3, 1).repeat(1, 1, 1, 3), fps=10)